In [1]:
# !pip install pandas
# !pip install torch
# !pip install tqdm  # pip install ipywidgets or something
# !pip install wandb

In [62]:
USING_WANDB = True  # Set to false if not Peter

if USING_WANDB:
    import wandb
    wandb.login()
    # !wandb login

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [73]:
import dna_dataset, constants, utils, ConvModel
import torch.nn as nn, torch.optim as optim, torch
from tqdm.notebook import tqdm
import importlib, os
from datetime import datetime

importlib.reload(dna_dataset)
importlib.reload(constants)
importlib.reload(utils)
importlib.reload(ConvModel)

<module 'ConvModel' from '/home/petern/Documents-WSL/dna-ml-model/ConvModel.py'>

In [8]:
# !mkdir Files
# !mv test.fasta Files
# !mv reduced_nonaccessible.fasta Files
# !mv accessible.fasta Files

Unzip the datafile

In [ ]:
!unzip $constants.DATA_ZIP_FILE

In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [36]:
full_dataset = dna_dataset.DNADataset(constants.ACCESSIBLE_FILE, constants.NOT_ACCESSIBLE_FILE)

In [10]:
# ensure the DNADataset is loaded properly
print('total sequences', len(full_dataset.sequences))
print('num accessible', full_dataset.accessible_count)
print('num not accessible', full_dataset.not_accessible_count)
i = 0
print(f"example entry {i}")
item = full_dataset[i]
print("label", item['labels'])
# print(item['sequences'])

# ensure dataset was shuffled properly
# check that not all the accessible labels are at the front
for i in range(full_dataset.accessible_count):
    if full_dataset[i]['labels'] != constants.ACCESSIBLE_LABEL:
        print('shuffled')
        break

total sequences 94478
num accessible 47239
num not accessible 47239
example entry 0
label 0
shuffled


In [37]:
# Split dataset
full_size = len(full_dataset)
val_size = round(constants.VALIDATION_SPLIT * full_size)
train_size = full_size - val_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print(len(train_dataset), len(val_dataset))
assert(len(train_dataset) + len(val_dataset) == full_size)

80306 14172


In [74]:
# insert torch model here, that takes sequence as input and output a label 0 or 1
model = ConvModel.CNNModel(kernel_size=2,
                           embed_dim=4,
                           num_filters1=128,
                           num_filters2=64,
                           pool_kernel_size=2,
                           hidden_dense1=128,
                           hidden_dense2=64,
                           dropout_rate_Dense=0.5
                           )
model.to(device)

CNNModel(
  (Conv1): Conv1d(4, 128, kernel_size=(2,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2): Conv1d(128, 64, kernel_size=(2,), stride=(1,), padding=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=3200, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=1, bias=True)
  (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout_Dense): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)

In [75]:
# parameters
epochs = constants.EPOCHS
batch_size = constants.BATCH_SIZE
learning_rate = constants.LEARNING_RATE
n_eval = constants.N_EVAL
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [76]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True
)

In [ ]:
# TRAINING LOOP
USING_WANDB = False
if USING_WANDB:

    note = "" #"0.5_dropout"
    wandb.init(
        project="dna_ml_model",
        # name=f"experiment_{run}"
        name=f"{note}colab_run2_balanced_data",
        config = {
            "architecture": model.__class__.__name__,
            "epochs": epochs,
            "learning_rate": learning_rate,
            "notes": note
    })


# step = 0
best_acc = 0.0


for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")

    running_loss = 0.0
    running_corrects = 0.0

    for batch in tqdm(train_loader):  # show the times for each batch
        # Forward propagate
        samples, labels = batch["sequences"].to(device), batch['labels'].to(device)

        outputs = model(samples)
        labels = labels.reshape(-1,1).float()

        # Backpropagation and gradient descent
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()  # reset gradients before next iteration

        running_loss += loss.item() * samples.size(0)  # loss per sample times batch size

        preds = torch.round(outputs)
        running_corrects += torch.sum(preds == labels.data).item()


    # if step % n_eval == 0:
    # Compute training loss and accuracy.
    with torch.no_grad():
        # Compute validation loss and accuracy.
        # accuracy = utils.compute_accuracy(outputs, labels)  # only does current batch
        val_loss, val_acc = utils.evaluate(val_loader, model, loss_fn, device)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects / len(train_dataset)
        print(f"Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
        print(f"Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # # deep copy the model
        # if val_accuracy > best_acc:
        #     best_acc = epoch_acc
        #     torch.save(model.state_dict(), best_model_params_path)

        if USING_WANDB:
            wandb.log({"Train Loss": epoch_loss,
                        "Train Acc": epoch_acc,
                        "Val Loss": val_loss,
                        "Val Acc": val_acc,
                        "Epoch": epoch
            })

        # step += 1

    print()

Save Model

In [31]:
# Create pretrained directory if not yet created
if not os.path.isdir(constants.PRETRAINED_DIR):
    os.mkdir(constants.PRETRAINED_DIR)

now = datetime.now()
datetime_str = now.strftime("%m-%d-%H-%M-%S")
model_save_path = os.path.join(
    constants.PRETRAINED_DIR,
    f'{datetime_str}-{model.__class__.__name__}-model-{epochs}-epochs.pt'
)
print('model_save_path', model_save_path)
ConvModel.save_CNNModel(model_save_path, model)
print(f"model saved at {datetime_str}")

model_save_path pretrained/12-10-05-59-29-CNNModel-model-100-epochs.pt
model saved at 12-10-05-59-29


Load Model

In [37]:
model_save_path = "/content/pretrained/12-10-05-30-17-CNNModel-model-10-epochs.pt"
model = ConvModel.load_CNNModel(model_save_path)
model.to(device)

CNNModel(
  (Conv1): Conv1d(4, 128, kernel_size=(2,), stride=(1,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2): Conv1d(128, 64, kernel_size=(2,), stride=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=3136, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
)

Inference

In [38]:
total_predictions = 0
total_correct = 0

for batch in tqdm(val_loader):
    val_samples, val_labels = batch['sequences'].to(device), batch['labels'].to(device)
    val_outputs = model(val_samples)
    # print(torch.round(val_outputs))
    val_labels = val_labels.reshape(-1, 1).float()

    val_loss = loss_fn(val_outputs, val_labels).item()  # change tensor to single val

    total_correct += (torch.round(val_outputs) == val_labels).sum().item()

    total_predictions += len(val_outputs)

val_accuracy = total_correct / total_predictions
print(f"Final Validation Accuracy: {val_accuracy * 100} %")

# wandb.summary['test_accuracy'] = val_accuracy

# wandb.finish()

  0%|          | 0/142 [00:00<?, ?it/s]

Final Validation Accuracy: 76.4817950889077 %


Inference on Test File

In [16]:
test_dataset = dna_dataset.TestDataset(constants.TEST_FILE) # TODO
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True
)
print("Test dataset loaded.")

Test dataset loaded.


In [ ]:
probs = []  # tuples of probability, id

for batch in tqdm(test_loader):

    samples, ids = batch["sequences"].to(device), batch['ids'] # not a tensor

    outputs = model(samples)

    out_list = outputs.tolist()

    for i in range(len(out_list)):
        probs.append((out_list[i], ids[i]))

print("Finished inference")

In [23]:
import numpy as np
np_probs = np.array(list(zip(*probs))[0])
print(len(np_probs[np_probs>0.5]), "true values out of ", len(np_probs), " total")  # assumes exactly 1
not_zero = np_probs[np_probs<=0.5]
not_zero.sort()
not_one = np_probs[np_probs>0.5]
not_one[::-1].sort()
print(not_zero)
print(not_one)

# print(np_probs[np_probs>0.0 and np_probs<1.0])

65517 true values out of  269315  total
[1.03285450e-10 1.81566789e-10 2.03635844e-10 ... 4.99967217e-01
 4.99980271e-01 4.99983966e-01]
[0.99994242 0.99988949 0.99988389 ... 0.50001734 0.50001717 0.5000127 ]


In [ ]:
probs.sort(reverse=True)

highest_probs = probs[:10000]  # top 10,000

with open(constants.SOLUTION_FILE, "w") as f:
    for pair in highest_probs:
        f.write(pair[1])
        # f.write("a")
        f.write("\n")

print(list(zip(*probs[:10]))[0])  # probs only

In [24]:
# zip_file_name = "predictions.zip"
# !zip $zip_file_name $constants.SOLUTION_FILE


  adding: predictions.csv (deflated 65%)


In [25]:
# ONLY for use on google colab. download files
from google.colab import files
import os
dir = 'pretrained'
model_file = os.path.join(dir, os.listdir(dir)[0])
files.download(model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(constants.SOLUTION_FILE)